In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import img_to_array, load_img
import numpy as np

Using TensorFlow backend.


In [12]:
###############################
#                             #
#          DEMO_04            #
#                             #
###############################

# 解bug
# Python：IOError: image file is truncated 的解决办法
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True # 解bug

#############################################################################################

try:
    train_data_dir = 'C:/Users/Java/Desktop/Project/Project_Image/data2/train'
    validation_data_dir = 'C:/Users/Java/Desktop/Project/Project_Image/data2/validation'
    nb_train_samples = 38000
    nb_validation_samples = 700
    epochs = 15 # 迭代
    batch_size = 100

    ##################################################################

    train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
            train_data_dir,
            target_size=(28, 28),
            batch_size=100,
            class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(
            validation_data_dir,
            target_size=(28, 28),
            batch_size=100,
            class_mode='categorical')

    ##################################################################

    print(train_generator.class_indices)  # 印出標籤
    print(validation_generator.class_indices)  # 印出標籤

    ##################################################################

    model = Sequential() # 建立model

    ##################################################################

    # 加入卷積層1
    model.add(Conv2D(filters=32, kernel_size = (3,3),   # 32個3*3濾鏡
                     input_shape = (28,28,3),   # 圖片大小&三原色
                     activation = 'relu',
                     padding = 'same' # 經過此層影像大小不變
                     ) 
             )


    # 控制overfit
    model.add(Dropout(rate=0.25))    


    # 加入池化層1
    model.add(MaxPooling2D(pool_size=(2,2))) 

    ##################################################################

    # 加入卷積層2
    model.add(Conv2D(filters=64, kernel_size = (3,3),   # 64個3*3濾鏡
                     activation = 'relu',
                     padding = 'same' # 經過此層影像大小不變
                     ) 
             )


    # 控制overfit
    model.add(Dropout(rate=0.25))          



    # 加入池化層2
    model.add(MaxPooling2D(pool_size=(2,2))) 

    ##################################################################

    # 加入平坦層
    model.add(Flatten()) #256*256*64


    # 控制overfit
    model.add(Dropout(rate=0.25))    


    ##################################################################

    # 加入隱藏層
    model.add(Dense(16384, activation = 'relu'))


    # 控制overfit
    model.add(Dropout(rate=0.25))    

    ##################################################################

    # 建立輸出層
    model.add(Dense(9, activation = 'softmax'))

    ##################################################################

    # 查看模型摘要
    print(model.summary())

    ##################################################################

    ###############################
    #                             #
    #          準備訓練            #
    #                             #
    ###############################
    model.compile(loss = 'categorical_crossentropy',  # 損失函數
                  optimizer = 'adam',                 # 最優化方法
                  metrics = ['accuracy'])             # 以準確率評估

##################################################################

    train_history = model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=15,
                        validation_data=validation_generator,
                        validation_steps= nb_validation_samples // batch_size)

    show_train_history(train_history,'acc','val_acc')
except:
    pass
#########################################################################################

# 過程問題: 建立資料集，輸入資料集，演算法選擇，層數問題，圖片大小問題，三原色... etc(待補)    
# 問題: 太慢                                                                            
# 問題: 演算法未優化                                                                      
# 問題: 只用了四種                                                                      
# 問題: size是否有問題                                                                  
# 問題: 迭代次數太少                                                                    
                                                                                    
#########################################################################################


Found 37936 images belonging to 9 classes.
Found 700 images belonging to 9 classes.
{'二條城': 1, '渡月橋': 7, '平等院鳳凰堂': 5, '京都塔': 2, '京都御所': 3, '千本鳥居': 4, '三十三間堂': 0, '金閣寺': 8, '清水寺': 6}
{'二條城': 1, '渡月橋': 7, '平等院鳳凰堂': 5, '京都塔': 2, '京都御所': 3, '千本鳥居': 4, '三十三間堂': 0, '金閣寺': 8, '清水寺': 6}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 28, 28, 32)        896       
_________________________________________________________________
dropout_41 (Dropout)         (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
dropout_42 (Dropout)         (None, 14, 14, 64)        0    

In [4]:
x_img_test_dir = 'C:/Users/Java/Desktop/Project/Project_Image/data/Predict'

def predict(basedir, model):
    print(train_generator.class_indices)  # 印出標籤列表
    for i in range(1,10):
        path = basedir + str(i) + '.jpg'
    
        img = load_img(path,False,target_size=(28,28))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        prediction = model.predict_classes(x)
        probs = model.predict_proba(x)
        print(probs)
        

        
        
print("===========二條城測試=============")
basedir = "C:/Users/Java/Desktop/Project/Project_Image/data/Predict/二條城/"
predict(basedir, model)
print("===========京都塔測試=============")
basedir = "C:/Users/Java/Desktop/Project/Project_Image/data/Predict/京都塔/"
predict(basedir, model)
print("==========千本鳥居測試==============")        
basedir = "C:/Users/Java/Desktop/Project/Project_Image/data/Predict/千本鳥居/"
predict(basedir, model)
print("===========金閣寺測試=============")
basedir = "C:/Users/Java/Desktop/Project/Project_Image/data/Predict/金閣寺/"
predict(basedir, model)

===========二條城測試=============
{'千本鳥居': 2, '金閣寺': 3, '京都塔': 1, '二條城': 0}
1/1 [==============================] - 0s
[[ 1.  0.  0.  0.]]
1/1 [==============================] - 0s
[[ 1.  0.  0.  0.]]
1/1 [==============================] - 0s
[[ 1.  0.  0.  0.]]
1/1 [==============================] - 0s
[[ 1.  0.  0.  0.]]
1/1 [==============================] - 0s
[[ 1.  0.  0.  0.]]
1/1 [==============================] - 0s
[[ 0.  1.  0.  0.]]
1/1 [==============================] - 0s
[[ 1.  0.  0.  0.]]
1/1 [==============================] - 0s
[[ 0.  0.  0.  1.]]
1/1 [==============================] - 0s
[[ 1.  0.  0.  0.]]
===========京都塔測試=============
{'千本鳥居': 2, '金閣寺': 3, '京都塔': 1, '二條城': 0}
1/1 [==============================] - 0s
[[ 1.  0.  0.  0.]]
1/1 [==============================] - 0s
[[ 0.  1.  0.  0.]]
1/1 [==============================] - 0s
[[ 0.  1.  0.  0.]]
1/1 [==============================] - 0s
[[ 0.  1.  0.  0.]]
1/1 [==============================] - 0s
[[ 0.  1

In [ ]:
###############################
#                             #
#          DEMO_03            #
#                             #
###############################

train_data_dir = 'C:/Users/Java/Desktop/Project/Project_Image/data/train'
validation_data_dir = 'C:/Users/Java/Desktop/Project/Project_Image/data/validation'
nb_train_samples = 900
nb_validation_samples = 400
epochs = 10 # 迭代
batch_size = 32 

##################################################################

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(28, 28),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(28, 28),
        batch_size=32,
        class_mode='categorical')

##################################################################

print(train_generator.class_indices)  # 印出標籤
print(validation_generator.class_indices)  # 印出標籤

##################################################################

model = Sequential() # 建立model

##################################################################

# 加入卷積層1
model.add(Conv2D(filters=32, kernel_size = (3,3),   # 32個3*3濾鏡
                 input_shape = (28,28,3),   # 圖片大小&三原色
                 activation = 'relu',
                 padding = 'same' # 經過此層影像大小不變
                 ) 
         )


# 控制overfit
model.add(Dropout(rate=0.25))    


# 加入池化層1
model.add(MaxPooling2D(pool_size=(2,2))) 

##################################################################

# 加入卷積層2
model.add(Conv2D(filters=64, kernel_size = (3,3),   # 64個3*3濾鏡
                 activation = 'relu',
                 padding = 'same' # 經過此層影像大小不變
                 ) 
         )


# 控制overfit
model.add(Dropout(rate=0.25))          



# 加入池化層2
model.add(MaxPooling2D(pool_size=(2,2))) 

##################################################################

# 加入平坦層
model.add(Flatten()) #256*256*64


# 控制overfit
model.add(Dropout(rate=0.25))    


##################################################################

# 加入隱藏層
model.add(Dense(16384, activation = 'relu'))


# 控制overfit
model.add(Dropout(rate=0.25))    

##################################################################

# 建立輸出層
model.add(Dense(4, activation = 'softmax'))

##################################################################

# 查看模型摘要
print(model.summary())

##################################################################

###############################
#                             #
#          準備訓練            #
#                             #
###############################
model.compile(loss = 'categorical_crossentropy',  # 損失函數
              optimizer = 'adam',                 # 最優化方法
              metrics = ['accuracy'])             # 以準確率評估

##################################################################

train_history = model.fit_generator(
                    train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=3000,
                    validation_data=validation_generator,
                    validation_steps= nb_validation_samples // batch_size)

show_train_history(train_history,'acc','val_acc')
#########################################################################################

# 過程問題: 建立資料集，輸入資料集，演算法選擇，層數問題，圖片大小問題，三原色... etc(待補)    
# 問題: 太慢                                                                            
# 問題: 演算法未優化                                                                      
# 問題: 只用了四種                                                                      
# 問題: size是否有問題                                                                  
# 問題: 迭代次數太少                                                                 
                                                                                    
#########################################################################################


Found 900 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
{'千本鳥居': 2, '金閣寺': 3, '京都塔': 1, '二條城': 0}
{'千本鳥居': 2, '金閣寺': 3, '京都塔': 1, '二條城': 0}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 32)        896       
_________________________________________________________________
dropout_5 (Dropout)          (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
dropout_6 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 

28/28 [==============================] - 24s - loss: 0.0299 - acc: 0.9877 - val_loss: 0.5358 - val_acc: 0.8940
Epoch 116/3000
28/28 [==============================] - 24s - loss: 0.0316 - acc: 0.9888 - val_loss: 0.3817 - val_acc: 0.9375
Epoch 117/3000
28/28 [==============================] - 24s - loss: 0.0068 - acc: 0.9978 - val_loss: 0.5255 - val_acc: 0.9022
Epoch 118/3000
28/28 [==============================] - 24s - loss: 0.0046 - acc: 0.9989 - val_loss: 0.4316 - val_acc: 0.9130
Epoch 119/3000
28/28 [==============================] - 25s - loss: 0.0182 - acc: 0.9933 - val_loss: 0.5030 - val_acc: 0.9212
Epoch 120/3000
28/28 [==============================] - 24s - loss: 0.0139 - acc: 0.9955 - val_loss: 0.6779 - val_acc: 0.8587
Epoch 121/3000
28/28 [==============================] - 24s - loss: 0.0191 - acc: 0.9933 - val_loss: 0.4595 - val_acc: 0.9321
Epoch 122/3000
28/28 [==============================] - 24s - loss: 0.0057 - acc: 0.9978 - val_loss: 0.5307 - val_acc: 0.9076
Epoch 1

28/28 [==============================] - 24s - loss: 0.0053 - acc: 0.9978 - val_loss: 0.6406 - val_acc: 0.9049
Epoch 246/3000
28/28 [==============================] - 24s - loss: 0.0115 - acc: 0.9966 - val_loss: 0.5218 - val_acc: 0.9103
Epoch 247/3000
28/28 [==============================] - 24s - loss: 0.0126 - acc: 0.9978 - val_loss: 0.5850 - val_acc: 0.9076
Epoch 248/3000
28/28 [==============================] - 24s - loss: 0.0084 - acc: 0.9966 - val_loss: 0.5361 - val_acc: 0.9130
Epoch 249/3000
28/28 [==============================] - 24s - loss: 0.0075 - acc: 0.9978 - val_loss: 0.5387 - val_acc: 0.9103
Epoch 250/3000
28/28 [==============================] - 24s - loss: 0.0305 - acc: 0.9955 - val_loss: 0.5971 - val_acc: 0.9049
Epoch 251/3000
28/28 [==============================] - 24s - loss: 0.0390 - acc: 0.9888 - val_loss: 0.6970 - val_acc: 0.8913
Epoch 252/3000
28/28 [==============================] - 24s - loss: 0.1010 - acc: 0.9899 - val_loss: 0.6143 - val_acc: 0.8940
Epoch 2

28/28 [==============================] - 24s - loss: 0.0183 - acc: 0.9955 - val_loss: 0.7488 - val_acc: 0.8859
Epoch 376/3000
28/28 [==============================] - 24s - loss: 0.0100 - acc: 0.9978 - val_loss: 0.6375 - val_acc: 0.9158
Epoch 377/3000
28/28 [==============================] - 25s - loss: 0.0149 - acc: 0.9989 - val_loss: 0.6066 - val_acc: 0.9293
Epoch 378/3000
28/28 [==============================] - 24s - loss: 0.0080 - acc: 0.9967 - val_loss: 0.6838 - val_acc: 0.9158
Epoch 379/3000
28/28 [==============================] - 25s - loss: 0.0101 - acc: 0.9978 - val_loss: 0.6488 - val_acc: 0.9049
Epoch 380/3000
28/28 [==============================] - 24s - loss: 0.0065 - acc: 0.9989 - val_loss: 0.6235 - val_acc: 0.9158
Epoch 381/3000
28/28 [==============================] - 24s - loss: 0.0083 - acc: 0.9966 - val_loss: 0.5520 - val_acc: 0.9239
Epoch 382/3000
28/28 [==============================] - 24s - loss: 0.0100 - acc: 0.9966 - val_loss: 0.6744 - val_acc: 0.9103
Epoch 3

28/28 [==============================] - 24s - loss: 0.0371 - acc: 0.9922 - val_loss: 0.7262 - val_acc: 0.8995
Epoch 441/3000
28/28 [==============================] - 24s - loss: 0.0114 - acc: 0.9955 - val_loss: 0.7731 - val_acc: 0.8913
Epoch 442/3000
28/28 [==============================] - 24s - loss: 0.0171 - acc: 0.9944 - val_loss: 0.8380 - val_acc: 0.8995
Epoch 443/3000
28/28 [==============================] - 24s - loss: 0.0307 - acc: 0.9966 - val_loss: 0.8076 - val_acc: 0.8940
Epoch 444/3000
28/28 [==============================] - 24s - loss: 0.0162 - acc: 0.9955 - val_loss: 0.8967 - val_acc: 0.9022
Epoch 445/3000
28/28 [==============================] - 24s - loss: 0.1043 - acc: 0.9866 - val_loss: 0.8442 - val_acc: 0.9103
Epoch 446/3000
28/28 [==============================] - 24s - loss: 0.0485 - acc: 0.9922 - val_loss: 0.8220 - val_acc: 0.8995
Epoch 447/3000
28/28 [==============================] - 24s - loss: 0.0358 - acc: 0.9933 - val_loss: 0.8957 - val_acc: 0.9076
Epoch 4

28/28 [==============================] - 24s - loss: 0.0771 - acc: 0.9944 - val_loss: 1.1692 - val_acc: 0.9022
Epoch 569/3000
28/28 [==============================] - 24s - loss: 0.0208 - acc: 0.9978 - val_loss: 1.2631 - val_acc: 0.8859
Epoch 570/3000
28/28 [==============================] - 24s - loss: 0.0305 - acc: 0.9978 - val_loss: 1.2982 - val_acc: 0.8859
Epoch 571/3000
28/28 [==============================] - 24s - loss: 0.0074 - acc: 0.9989 - val_loss: 1.0561 - val_acc: 0.8913
Epoch 572/3000
28/28 [==============================] - 24s - loss: 0.0069 - acc: 0.9989 - val_loss: 1.1492 - val_acc: 0.8886
Epoch 573/3000
28/28 [==============================] - 24s - loss: 0.0033 - acc: 0.9989 - val_loss: 1.1217 - val_acc: 0.8995
Epoch 574/3000
28/28 [==============================] - 24s - loss: 0.0078 - acc: 0.9989 - val_loss: 1.0139 - val_acc: 0.9049
Epoch 575/3000
28/28 [==============================] - 24s - loss: 0.0308 - acc: 0.9966 - val_loss: 1.3167 - val_acc: 0.8641
Epoch 5

28/28 [==============================] - 24s - loss: 0.0512 - acc: 0.9911 - val_loss: 1.1601 - val_acc: 0.8995
Epoch 697/3000
28/28 [==============================] - 24s - loss: 0.0467 - acc: 0.9955 - val_loss: 1.0465 - val_acc: 0.9103
Epoch 698/3000
28/28 [==============================] - 24s - loss: 0.0194 - acc: 0.9978 - val_loss: 0.9801 - val_acc: 0.9185
Epoch 699/3000
28/28 [==============================] - 24s - loss: 0.0177 - acc: 0.9989 - val_loss: 0.9227 - val_acc: 0.9130
Epoch 700/3000
28/28 [==============================] - 24s - loss: 0.0049 - acc: 0.9989 - val_loss: 1.0006 - val_acc: 0.9103
Epoch 701/3000
28/28 [==============================] - 24s - loss: 0.0181 - acc: 0.9989 - val_loss: 1.2035 - val_acc: 0.9103
Epoch 702/3000
28/28 [==============================] - 24s - loss: 0.0017 - acc: 0.9989 - val_loss: 1.0610 - val_acc: 0.9130
Epoch 703/3000
28/28 [==============================] - 24s - loss: 1.6951e-06 - acc: 1.0000 - val_loss: 1.0874 - val_acc: 0.9185
Epo

28/28 [==============================] - 25s - loss: 1.1974e-07 - acc: 1.0000 - val_loss: 1.2425 - val_acc: 0.9022
Epoch 825/3000
28/28 [==============================] - 25s - loss: 2.4305e-05 - acc: 1.0000 - val_loss: 1.2337 - val_acc: 0.9049
Epoch 826/3000
28/28 [==============================] - 25s - loss: 0.0041 - acc: 0.9989 - val_loss: 1.0627 - val_acc: 0.9185
Epoch 827/3000
28/28 [==============================] - 25s - loss: 0.0056 - acc: 0.9978 - val_loss: 1.0926 - val_acc: 0.9130
Epoch 828/3000
28/28 [==============================] - 25s - loss: 0.0034 - acc: 0.9989 - val_loss: 1.0793 - val_acc: 0.9103
Epoch 829/3000
 9/28 [========>.....................] - ETA: 16s - loss: 0.0508 - acc: 0.9931